In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
#import seaborn as sns
#%pylab inline

In [82]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-02-17/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 3281
files_end = 3476

In [99]:
# cup configuration
cup_config = cycle_Sb_2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125", "Xe" : "129"}
#corr_isotopes_Sb = {"Te": "125"}
#
# Mass Range of cup configuration
mass_range = cycle_Sb_2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"], ["117", "118", "119", "122", "124"]]
#isotopes = [["112", "114", "115", "116", "117", "118", "119", "122", "124"]]
isotopes = [["116", "117", "119", "120", "122", "124"]]
isotopes_column = ["116", "117", "119", "120", "122", "124"]
data_sample_column = [(i + "/" + "118") for i in isotopes_column]

denom_isotope = "118"
norm_ratio = ["122","118"]
norm_ratio_Sb= ["123","121"]
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False

blk_corr = True

In [89]:
columns = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)"]
blk_ls = []
sample_ls = []
df_all = pd.DataFrame()
#files_1 = range(files_start, files_end, 1)
files_1 = range(3281, 3443) + range(3445, 3476)
for sample in files_1:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero["sample"] = sample_name
    df_zero["cycle"] = df_zero.index.values
    df_zero.index = df_zero["date"]
    df_zero = df_zero[columns]
    df_all = df_all.append(df_zero, ignore_index = True)

    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "teflon blk" or sample_name == "blank 3_1" or sample_name == "col blk 271115":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1)
date,,,,,,,,,,,,,,
2016-02-18 15:29:00,1,blank sol,0.015104,0.015905,3.557100e-06,0.000392,0.002808,0.000254,0.003620,0.002831,0.000502,0.001381,0.000485,0.001040
2016-02-18 15:29:00,2,blank sol,0.015130,0.015927,1.493920e-05,0.000385,0.002822,0.000263,0.003682,0.002941,0.000529,0.001479,0.000489,0.001080
2016-02-18 15:29:00,3,blank sol,0.015223,0.016083,2.774400e-05,0.000468,0.002897,0.000314,0.003823,0.003153,0.000585,0.001592,0.000518,0.001176
2016-02-18 15:29:00,4,blank sol,0.015288,0.016139,4.908540e-05,0.000368,0.002805,0.000273,0.003629,0.002932,0.000521,0.001482,0.000515,0.001090
2016-02-18 15:29:00,5,blank sol,0.015172,0.016012,9.959400e-06,0.000486,0.002848,0.000308,0.003738,0.003223,0.000628,0.001683,0.000561,0.001191
2016-02-18 15:29:00,6,blank sol,0.015269,0.016062,1.422770e-05,0.000404,0.002808,0.000275,0.003644,0.002952,0.000525,0.001447,0.000495,0.001115
2016-02-18 15:29:00,7,blank sol,0.015242,0.016039,1.565030e-05,0.000394,0.002786,0.000265,0.003633,0.002839,0.000491,0.001425,0.000460,0.001033
2016-02-18 15:29:00,8,blank sol,0.015224,0.015997,3.556900e-05,0.000387,0.002776,0.000305,0.003641,0.002938,0.000530,0.001471,0.000511,0.001065
2016-02-18 15:29:00,9,blank sol,0.015218,0.016048,2.774400e-05,0.000367,0.002657,0.000245,0.003492,0.002766,0.000478,0.001325,0.000460,0.001013


In [100]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()
df_external_outlier = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)

# arbitrary blank positions
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True
    
#blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False    
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    #data_sample = new_corr.external_norm_Sb(norm_ratio_Sb, denom_isotope, iter_beta)
    data_sample = new_corr.internal_norm_1(norm_ratio, denom_isotope, iter_beta)
# new method for df with all data together with signals in pycharm
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = new_corr.mad_outlier_rejection(data_sample)
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample = data_sample[sorted(data_sample.columns)]
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    #data_sample_outlier = data_sample_outlier[["116", "117", "119", "120", "122", "124"]]
    #data_sample_outlier.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    #data_sample_outlier["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    #data_sample_outlier["sample"] = sample_name
    #data_sample_outlier["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)
    #df_external_outlier = df_external_outlier.append(data_sample_outlier) 
    print sample, sample_name

3282 NIST 100ppb
3283 NIST 100ppb
3284 NIST 100ppb
3285 NIST 100ppb
3286 NIST 100ppb
3288 NIST 100ppb
3289 NIST 100ppb
3290 NIST 100ppb
3291 NIST 100ppb
3292 NIST 100ppb
3294 NIST 100ppb
3295 SPEX 100ppb
3296 NIST 100ppb
3297 SPEX 100ppb
3298 NIST 100ppb
3300 NIST 100ppb
3301 SPEX 100ppb
3302 NIST 100ppb
3303 SPEX 100ppb
3304 NIST 100ppb
3306 NIST 100ppb
3307 J2 4_1 I
3308 NIST 100ppb
3309 ZH09-05 50.5_1(2) I
3310 NIST 100ppb
3312 NIST 100ppb
3313 ZH09-05 56.7_1(2) I
3314 NIST 100ppb
3315 ZH09-05 44.8_1(2) I
3316 NIST 100ppb
3318 NIST 100ppb
3319 NIST 100ppb
3320 NIST 100ppb
3321 NIST 100ppb
3322 NIST 100ppb
3324 NIST 100ppb
3325 ZH09-05 23.3_2_1(1) I
3326 NIST 100ppb
3327 ZH09-05 10.3_1(2) I
3328 NIST 100ppb
3330 NIST 100ppb
3331 NIST 5_1 I
3332 NIST 100ppb
3333 J2 3_1(1) I
3334 NIST 100ppb
3336 NIST 100ppb
3337 NIST 100ppb
3338 NIST 100ppb
3339 NIST 100ppb
3340 NIST 100ppb
3342 NIST 100ppb
3343 ZH09-05 50.5_1(1) I
3344 NIST 100ppb
3345 ZH09-05 56.7_1(1) I
3346 NIST 100ppb
3348 NIST 1

In [69]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()
df_external_outlier = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # method for blank correction
    # arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample == True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False  
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
    
    data_sample_raw = new_corr.raw_signals("118")
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = data_sample_raw[np.abs(data_sample_raw-data_sample_raw.mean())<=(2*data_sample_raw.std())]
    
    data_sample = new_corr.external_norm_Sb("118", 10)
# new method for df with all data together with signals in pycharm
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = data_sample[np.abs(data_sample-data_sample.mean())<=(2*data_sample.std())]
    data_sample_outlier= data_sample[np.abs(data_sample-data_sample.mean())>(2*data_sample.std())]
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample[["116", "117", "119", "120", "122", "124"]]
    data_sample.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    data_sample_outlier = data_sample_outlier[["116", "117", "119", "120", "122", "124"]]
    data_sample_outlier.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    data_sample_outlier["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample_outlier["sample"] = sample_name
    data_sample_outlier["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)
    df_external_outlier = df_external_outlier.append(data_sample_outlier) 

In [101]:
df_external_norm.set_index("date", inplace = True)
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_external_sub = pd.DataFrame(df_external_norm, columns = columns)

df_all_sub = df_all[(df_all["sample"] != "blank sol") & (df_all["sample"] != "teflon blk") & (sample_name != "blank 3_1") & (sample_name != "col blk 271115") & (df_all["sample"] != "SQ")]
df_external_sub["125"] = df_all_sub["H6 (1)"]
df_external_sub["Te/Sn"] = (df_all_sub["H6 (1)"]/7.0509)/(df_all_sub["L1 (1)"]/24.223)

#df_external_sub = df_external_sub[df_external_sub["118"] > 4]
df_external_sub

,cycle,sample,116/118,117/118,119/118,120/118,122/118,124/118,116,117,118,119,120,122,124,125,Te/Sn
date,,,,,,,,,,,,,,,,,
2016-02-18 15:36:00,1,NIST 100ppb,0.600285,0.316698,0.354633,1.346254,NaN,0.239045,5.058576,2.703009,8.643133,3.103772,11.929370,1.736729,2.223737,-1.150060e-05,-4.570494e-06
2016-02-18 15:36:00,2,NIST 100ppb,0.600265,0.316694,0.354650,1.346131,NaN,0.239046,5.130494,2.741483,8.766252,3.148130,12.098121,1.761445,2.255381,-1.505760e-05,-5.900001e-06
2016-02-18 15:36:00,3,NIST 100ppb,0.600243,0.316718,0.354630,1.346152,0.19125,0.239030,5.169573,2.762580,8.832731,3.171715,12.189214,1.774564,2.271875,-1.292340e-05,-5.025635e-06
2016-02-18 15:36:00,4,NIST 100ppb,0.600235,0.316722,0.354625,1.346005,0.19125,0.238982,5.218165,2.788645,8.916074,3.201630,12.303154,1.791375,2.292977,-2.146010e-05,-8.267449e-06
2016-02-18 15:36:00,5,NIST 100ppb,0.600218,0.316681,0.354627,1.346158,NaN,0.239055,5.218399,2.788403,8.916179,3.201589,12.303960,1.791198,2.293330,9.129500e-06,3.517032e-06
2016-02-18 15:36:00,6,NIST 100ppb,0.600168,0.316710,0.354631,1.345989,0.19125,0.239037,5.274713,2.819140,9.014118,3.236970,12.438858,1.811247,2.319058,-1.078930e-05,-4.111329e-06
2016-02-18 15:36:00,7,NIST 100ppb,0.600277,0.316718,0.354618,1.346186,0.19125,0.239034,5.149389,2.751652,8.797864,3.159108,12.141557,1.767605,2.263037,-1.007770e-05,-3.934550e-06
2016-02-18 15:36:00,8,NIST 100ppb,0.600266,0.316693,0.354606,1.346094,0.19125,0.239027,5.200133,2.778654,8.885070,3.190370,12.261540,1.785258,2.285652,-7.232400e-06,-2.795984e-06
2016-02-18 15:36:00,9,NIST 100ppb,0.600233,0.316699,0.354608,1.346057,0.19125,0.239027,5.257649,2.809551,8.983527,3.225693,12.396658,1.804910,2.310731,-2.928520e-05,-1.119742e-05


In [102]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

df_external_norm_mean = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).mean()
df_external_norm_mean_sd = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).agg([np.mean, SE])
df_external_norm_mean["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]
df_external_norm_mean_sd["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]

#df_external_norm_mean = df_external_norm_mean[1:]
#df_external_norm_mean = df_external_norm_mean.drop(datetime.strptime("20150310 23:59", "%Y%m%d %H:%M"))
#df_external_norm_mean_sd = df_external_norm_mean_sd[1:]
df_external_norm_mean

,116/118,117/118,119/118,120/118,122/118,124/118,116,117,118,119,120,122,124,125,Te/Sn,sample
date,,,,,,,,,,,,,,,,
2016-02-18 15:36:00,0.600243,0.316707,0.354626,1.346138,0.19125,0.239028,5.215762,2.787223,8.912021,3.200220,12.299020,1.790645,2.292537,-7.659178e-06,-2.959734e-06,NIST 100ppb
2016-02-18 15:44:00,0.600249,0.316710,0.354633,1.346113,0.19125,0.239019,5.082144,2.715840,8.683736,3.118321,11.983890,1.744816,2.233802,-8.631493e-06,-3.410085e-06,NIST 100ppb
2016-02-18 15:53:00,0.600255,0.316714,0.354631,1.346120,0.19125,0.239015,5.207605,2.782869,8.897897,3.195185,12.279337,1.787798,2.288763,-1.141173e-05,-4.398011e-06,NIST 100ppb
2016-02-18 16:02:00,0.600248,0.316710,0.354633,1.346098,0.19125,0.239012,5.272701,2.817639,9.009143,3.235124,12.432554,1.810120,2.317295,-8.987152e-06,-3.408576e-06,NIST 100ppb
2016-02-18 16:11:00,0.600250,0.316714,0.354631,1.346109,0.19125,0.239008,5.061945,2.705058,8.649082,3.105839,11.935863,1.737804,2.224696,-6.479500e-06,-2.584795e-06,NIST 100ppb
2016-02-18 16:26:00,0.600249,0.316713,0.354631,1.346103,0.19125,0.239007,5.056467,2.702007,8.639008,3.102091,11.920933,1.735501,2.221566,-1.019585e-06,-4.098651e-07,NIST 100ppb
2016-02-18 16:35:00,0.600253,0.316720,0.354635,1.346107,0.19125,0.239011,5.104839,2.727992,8.722218,3.132119,12.036636,1.752462,2.243462,-1.339755e-06,-5.345239e-07,NIST 100ppb
2016-02-18 16:44:00,0.600250,0.316716,0.354637,1.346097,0.19125,0.239009,5.283823,2.823615,9.027972,3.241905,12.458235,1.813830,2.321954,-5.264272e-06,-2.007379e-06,NIST 100ppb
2016-02-18 16:52:00,0.600259,0.316719,0.354636,1.346106,0.19125,0.239006,5.076027,2.712519,8.672656,3.114280,11.967906,1.742402,2.230475,-1.464272e-06,-5.593109e-07,NIST 100ppb


In [11]:
plot_Nist = df_external_norm_mean[df_external_norm_mean["sample"] == "NIST 100ppb"]
plot_Nist = df_external_norm_mean[(df_external_norm_mean["118"] >=1)]
#dataplot = plot(plot_Nist["Te/Sn"], plot_Nist["124/120"], "ro")
dataplot = plot(plot_Nist.index, plot_Nist["122/120"], "ro")
show(dataplot)

In [78]:
df_external_norm_mean = df_external_norm_mean[(df_external_norm_mean["118"] >=1)]
sub_index = df_external_norm_mean[(df_external_norm_mean["sample"].shift(1) == df_external_norm_mean["sample"].shift(-1)) & (df_external_norm_mean["sample"].shift(1) == "NIST 100ppb")].index
sample = df_external_norm_mean["sample"]
new = df_external_norm_mean[df_external_norm_mean.columns[df_external_norm_mean.dtypes != 'object']]

new =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*10000
new["sample"] = sample

new = new[new.index.isin(sub_index)]
#df_external_norm_signal = df_external_norm_signal[df_external_norm_signal['sample'].isin(sub_index.tolist())]
#df_new_signal = new.merge(df_external_norm_signal, left_index=True, right_index=True)
#new[new.sample != "NIST 100ppb"]
new

KeyError: '118'

In [37]:
def SD2(x):
    SD2 = np.std(x) * 2
    return SD2

#new = new.drop(["116", "117", "118", "119", "120", "122", "124"], axis=1)
np.round(new.groupby("sample").agg([np.mean, SD2]), decimals=2)

116/120       117/120       118/120       119/120        \
                        mean   SD2    mean   SD2    mean   SD2    mean   SD2   
sample                                                                         
J2 2_1(2)               0.20  0.00   -0.13  0.00   -0.02  0.00    0.13  0.00   
J2 3_1(1)              -6.07  0.00   -6.13  0.00   -4.73  0.00   -2.26  0.00   
Jenny 12-06W 57_1(1)   -0.16  0.00   -0.25  0.00    0.06  0.00   -0.05  0.00   
NIST 100ppb             0.02  0.56    0.01  0.51    0.00  0.28    0.01  0.29   
NIST 3_1               -0.15  0.00   -0.17  0.00   -0.03  0.00    0.10  0.00   
NIST 4_1                0.08  0.00    0.10  0.00    0.02  0.00   -0.01  0.00   
SPEX 100ppb            -0.17  0.73   -0.10  0.60   -0.07  0.41   -0.08  0.30   
ZH09-05 10.3_1(1)      -0.06  0.00   -0.22  0.00    0.05  0.00   -0.16  0.00   
ZH09-05 23.3_1_1(1)     3.27  0.00   -0.17  0.00    0.04  0.00   -0.12  0.00   
ZH09-05 23.3_1_2(1)     0.47  0.00   -0.00  0.00    0.06  0.00   -0.06  0.00   
ZH09-05 33.8_1(1)       0.51  0.00   -0.10  0.00    0.07  0.00    0.06  0.00   
ZH09-05 44.8_1(1)       0.03  0.00   -0.00  0.00    0.07  0.00   -0.03  0.00   
ZH09-05 50.5_1(1)      -0.28  0.00   -0.38  0.00   -0.02  0.00   -0.03  0.00   
ZH09-05 56.7_1(1)      -0.33  0.00   -0.36  0.00   -0.10  0.00   -0.04  0.00   

                     122/120       ...        118              119          \
                        mean SD2   ...       mean     SD2     mean     SD2   
sample                             ...                                       
J2 2_1(2)                  0   0   ...     642.73    0.00   643.74    0.00   
J2 3_1(1)                  0   0   ...   -7958.50    0.00 -7958.07    0.00   
Jenny 12-06W 57_1(1)       0   0   ...     898.92    0.00   899.41    0.00   
NIST 100ppb                0   0   ...      -6.10  352.01    -6.21  351.58   
NIST 3_1                   0   0   ...     924.77    0.00   924.69    0.00   
NIST 4_1                   0   0   ...     859.77    0.00   858.88    0.00   
SPEX 100ppb                0   0   ...      52.85  359.93    53.27  361.19   
ZH09-05 10.3_1(1)          0   0   ...     939.29    0.00   941.93    0.00   
ZH09-05 23.3_1_1(1)        0   0   ...     978.70    0.00   986.36    0.00   
ZH09-05 23.3_1_2(1)        0   0   ...     825.47    0.00   825.31    0.00   
ZH09-05 33.8_1(1)          0   0   ...    1095.53    0.00  1095.95    0.00   
ZH09-05 44.8_1(1)          0   0   ...     749.95    0.00   748.65    0.00   
ZH09-05 50.5_1(1)          0   0   ...    1959.29    0.00  1958.85    0.00   
ZH09-05 56.7_1(1)          0   0   ...     947.34    0.00   947.97    0.00   

                          120              122              124          
                         mean     SD2     mean     SD2     mean     SD2  
sample                                                                   
J2 2_1(2)              644.46    0.00   646.13    0.00   647.78    0.00  
J2 3_1(1)            -7957.64    0.00 -7957.73    0.00 -7957.00    0.00  
Jenny 12-06W 57_1(1)   900.09    0.00   901.42    0.00   902.63    0.00  
NIST 100ppb             -6.02  352.60    -6.15  352.08    -6.04  352.79  
NIST 3_1               924.36    0.00   923.92    0.00   923.51    0.00  
NIST 4_1               858.04    0.00   856.36    0.00   854.59    0.00  
SPEX 100ppb             54.39  360.33    56.63  360.30    57.69  361.75  
ZH09-05 10.3_1(1)      944.91    0.00   950.51    0.00   956.17    0.00  
ZH09-05 23.3_1_1(1)    994.25    0.00  1009.61    0.00  1024.79    0.00  
ZH09-05 23.3_1_2(1)    825.31    0.00   825.18    0.00   824.86    0.00  
ZH09-05 33.8_1(1)     1096.28    0.00  1097.13    0.00  1097.58    0.00  
ZH09-05 44.8_1(1)      747.59    0.00   745.41    0.00   742.97    0.00  
ZH09-05 50.5_1(1)     1958.45    0.00  1957.66    0.00  1951.36    0.00  
ZH09-05 56.7_1(1)      948.59    0.00   949.70    0.00   951.00    0.00  

[14 rows x 26 columns]

In [93]:
df_all.to_csv(path_sav + "Sn_H8_L3_Sb_NU_Signals_zero_corr.csv", mode='a', header=False)

In [100]:
df_external_sub.to_csv(path_sav + "Sn_H6_L3_SB_external_norm.csv", mode='a', header=False)

In [103]:
df_external_norm_mean_sd.to_csv(path + "Sn_H8_L3_SB_internal_norm_122_118_MAD_bgd_corr.csv", mode='w', header=True)